In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

%tensorflow_version 1.15.0
import tensorflow as tf
import warnings
import math
import sys
import holidays
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import WDL,xDeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,get_feature_names

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from sklearn import metrics

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from math import sqrt

import warnings
warnings.filterwarnings('ignore')

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
pip install deepctr

     |████████████████████████████████| 147 kB 17.8 MB/s 
     |████████████████████████████████| 2.9 MB 36.7 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [2]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

rm: cannot remove '/root/.kaggle': No such file or directory
ref                                                                            title                                             size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
datasets/muratkokludataset/acoustic-extinguisher-fire-dataset                  Acoustic Extinguisher Fire Dataset               621KB  2022-04-02 22:59:36            921        642  0.9375           
datasets/muratkokludataset/rice-image-dataset                                  Rice Image Dataset                               219MB  2022-04-03 02:12:00           1006        592  0.875            
datasets/mdmahmudulhasansuzan/students-adaptability-level-in-online-education  Students Adaptability Level in Online Education    6KB  2022

In [4]:
!kaggle competitions download -c expedia-hotel-recommendations
! mkdir train

 99% 677M/686M [00:07<00:00, 147MB/s]
100% 686M/686M [00:07<00:00, 98.8MB/s]


In [5]:
! unzip expedia-hotel-recommendations.zip -d train

Archive:  expedia-hotel-recommendations.zip
  inflating: train/destinations.csv  
  inflating: train/sample_submission.csv  
  inflating: train/test.csv          
  inflating: train/train.csv         


In [6]:
#Reading the data from train and destination csv
dataframe = pd.read_csv('train/train.csv', sep=',', nrows=100000)
destinations = pd.read_csv('train/destinations.csv', sep=',')
dataframe.shape

(100000, 24)

In [7]:
#Renaming columns hotel_cluster to item id and isbooking to rating
dataframe = dataframe.rename(columns={'hotel_cluster': 'item_id', 'is_booking': 'rating'})

In [8]:
#describing the data
dataframe.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,rating,cnt,hotel_continent,hotel_country,hotel_market,item_id
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


# Feature Engineering


In [9]:
dataframe= dataframe.drop(['orig_destination_distance'],axis=1)
dataframe = dataframe.dropna()

In [10]:
#Getting USA holidays and Canada holidays
ca_holidays = holidays.Canada()
us_holidays = holidays.UnitedStates()

uk_holidays = holidays.UnitedKingdom()
gr_holidays = holidays.Germany()

In [11]:
# checking if check-in data or checkout date is a holiday in different countries
#Adding a column north_am_ci giving it a value of 1 if the check in date is a holiday
dataframe['north_am_ci'] = dataframe['srch_ci'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
#Adding a column north_am_co giving it a value of 1 if the check in date is a holiday
dataframe['north_am_co'] = dataframe['srch_co'].apply(lambda x: 1 if x in (us_holidays or ca_holidays)  else 0)
#Adding a column europe_ci giving it a value of 1 if the check in date is a holiday
dataframe['europe_ci'] = dataframe['srch_ci'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)
#Adding a europe_co giving it a value of 1 if the check in date is a holiday
dataframe['europe_co'] = dataframe['srch_co'].apply(lambda x: 1 if x in (uk_holidays or gr_holidays)  else 0)

In [12]:
dataframe.head(n=2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,user_id,is_mobile,is_package,channel,...,rating,cnt,hotel_continent,hotel_country,hotel_market,item_id,north_am_ci,north_am_co,europe_ci,europe_co
0,2014-08-11 07:46:59,2,3,66,348,48862,12,0,1,9,...,0,3,2,50,628,1,0,0,0,0
1,2014-08-11 08:22:12,2,3,66,348,48862,12,0,1,9,...,1,1,2,50,628,1,0,0,0,0


In [13]:
# extracting month from date_time
dataframe['click_month'] = pd.DatetimeIndex(dataframe['date_time']).month
dataframe['checkin_month'] = pd.DatetimeIndex(dataframe['srch_ci']).month
dataframe['checkout_month'] = pd.DatetimeIndex(dataframe['srch_co']).month

In [14]:
#Removing date-time and checkin and check out columns from datafram
dataframe= dataframe.drop(['date_time'],axis=1)
dataframe= dataframe.drop(['srch_ci'],axis=1)
dataframe= dataframe.drop(['srch_co'],axis=1)

In [15]:
dataframe.shape

(99929, 27)

In [16]:
dataframe.columns

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city', 'user_id', 'is_mobile',
       'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt',
       'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id',
       'rating', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'item_id', 'north_am_ci', 'north_am_co', 'europe_ci', 'europe_co',
       'click_month', 'checkin_month', 'checkout_month'],
      dtype='object')

In [17]:
# categ_sparse / conti_dense
sparse_features = ['site_name','posa_continent','user_location_country','user_location_region','user_location_city',
             'user_id','is_mobile','is_package','channel','click_month','checkin_month','checkout_month',
            'srch_adults_cnt','srch_children_cnt','srch_rm_cnt','srch_destination_id','hotel_continent',
               'hotel_country','cnt','north_am_ci', 'north_am_co', 'europe_ci', 'europe_co','click_month',
                   'checkin_month','checkout_month']

dense_features = ['hotel_market']
target = ['rating']

## Label Encoding for Sparse Features and Normalization For Dense Features


In [18]:
# Performing Label encoding for the sparse features
for feature in sparse_features:
    lbe = LabelEncoder()
    dataframe[feature] = lbe.fit_transform(dataframe[feature])

In [19]:
# Performing min-max scaling for the dense features
mms = MinMaxScaler(feature_range=(0,1))
dataframe[dense_features] = mms.fit_transform(dataframe[dense_features])

### Generate feature columns
For sparse features, we transform them into dense vectors by embedding techniques. For dense numerical features, we concatenate them to the input tensors of fully connected layer.

In [20]:
# count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, dataframe[feat].nunique(),embedding_dim=4)
                          for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
#Split the model into train and test dataset
train, test = train_test_split(dataframe, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

### MOdel before Hyper paramter Tuning

In [ ]:
model = xDeepFM(linear_feature_columns,dnn_feature_columns,task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'])

history = model.fit(train_model_input, train[target].values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 50s - loss: 0.2730 - binary_crossentropy: 0.2728 - val_loss: 0.2472 - val_binary_crossentropy: 0.2467
Epoch 2/10
63954/63954 - 50s - loss: 0.2219 - binary_crossentropy: 0.2211 - val_loss: 0.2574 - val_binary_crossentropy: 0.2563
Epoch 3/10
63954/63954 - 49s - loss: 0.2064 - binary_crossentropy: 0.2050 - val_loss: 0.2750 - val_binary_crossentropy: 0.2733
Epoch 4/10
63954/63954 - 49s - loss: 0.1970 - binary_crossentropy: 0.1951 - val_loss: 0.2853 - val_binary_crossentropy: 0.2831
Epoch 5/10
63954/63954 - 49s - loss: 0.1909 - binary_crossentropy: 0.1885 - val_loss: 0.3121 - val_binary_crossentropy: 0.3096
Epoch 6/10
63954/63954 - 50s - loss: 0.1860 - binary_crossentropy: 

In [ ]:
auc = roc_auc_score(test[target].values, pred_ans)

print("Root-mean-square:\t%f" % np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3))

Root-mean-square:	0.295000


### To get best parameters 


In [ ]:
#Assume certain parameters in a dictionary to perform hyper parameter tuning to get the best parameters
param_rand = {'dnn_hidden_units' : [(1,1),(2,2),(4,4),(8,8),(29,29),(128,128),(256,256)],
              'cin_layer_size':[(1,1),(2,2),(4,4),(8,8),(32,32),(128,128)],
              'cin_split_half':[True,False],
              'dnn_use_bn':[True,False],
              'l2_reg_cin':[1e-7,1e-5,1e-3,1e-1,1],
              'l2_reg_linear':[1e-5,1e-3,1e-1,1,10],
              'l2_reg_embedding':[1e-7,1e-5,1e-3,1e-1,1],
              'l2_reg_dnn':[0,0.2,2,4],
              'dnn_dropout':[0, 0.3, 0.5, 0.7, 0.9]
             }
result = {}

### To find the best parameter for DNN_Hidden_Unit

In [ ]:
dnn_hidden_units_dict = {}
for i in param_rand['dnn_hidden_units']:
    model = xDeepFM(linear_feature_columns,dnn_feature_columns, dnn_hidden_units=i, 
              seed=1024,task='binary')
    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    dnn_hidden_units_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['dnn_hidden_units']=dnn_hidden_units_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 49s - loss: 0.0810 - mean_squared_error: 0.0808 - val_loss: 0.0710 - val_mean_squared_error: 0.0706
Epoch 2/10
63954/63954 - 50s - loss: 0.0664 - mean_squared_error: 0.0658 - val_loss: 0.0726 - val_mean_squared_error: 0.0718
Epoch 3/10
63954/63954 - 48s - loss: 0.0631 - mean_squared_error: 0.0622 - val_loss: 0.0760 - val_mean_squared_error: 0.0750
Epoch 4/10
63954/63954 - 48s - loss: 0.0606 - mean_squared_error: 0.0595 - val_loss: 0.0780 - val_mean_squared_error: 0.0768
Epoch 5/10
63954/63954 - 49s - loss: 0.0589 - mean_squared_error: 0.0576 - val_loss: 0.0808 - val_mean_squared_error: 0.0795
Epoch 6/10
63954/63954 - 48s - loss: 0.0575 - mean_squared_error: 0.0561 - val_loss: 0.0839 - val_mean_squared_error: 0.0825
Epoch 7/10
63954/63954 - 49s - loss: 0.0564 - mean_squared_error: 0.0549 - val_loss: 0.0868 - val_mean_squared_error: 0.0853
Epoch 8/10
63954/63954 - 49s - loss: 0.0554 - mean_squared_error: 0.0539 - 

In [ ]:
dnn_hidden_units_dict

{(1, 1): {'RMSE': 0.284},
 (2, 2): {'RMSE': 0.291},
 (4, 4): {'RMSE': 0.294},
 (8, 8): {'RMSE': 0.295},
 (29, 29): {'RMSE': 0.289},
 (128, 128): {'RMSE': 0.284},
 (256, 256): {'RMSE': 0.296}}

### To get best parameter cin_layer_size

In [ ]:
cin_layer_size_dict = {}
for i in param_rand['cin_layer_size']:
    model= xDeepFM(linear_feature_columns,dnn_feature_columns,cin_layer_size=i,dnn_hidden_units=(256, 256),
                   cin_activation='relu', seed=1024,task='binary',cin_split_half=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    cin_layer_size_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['cin_layer_size']=cin_layer_size_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 11s - loss: 0.0796 - mean_squared_error: 0.0794 - val_loss: 0.0707 - val_mean_squared_error: 0.0704
Epoch 2/10
63954/63954 - 7s - loss: 0.0657 - mean_squared_error: 0.0652 - val_loss: 0.0729 - val_mean_squared_error: 0.0723
Epoch 3/10
63954/63954 - 7s - loss: 0.0623 - mean_squared_error: 0.0615 - val_loss: 0.0755 - val_mean_squared_error: 0.0746
Epoch 4/10
63954/63954 - 7s - loss: 0.0602 - mean_squared_error: 0.0592 - val_loss: 0.0786 - val_mean_squared_error: 0.0776
Epoch 5/10
63954/63954 - 7s - loss: 0.0586 - mean_squared_error: 0.0576 - val_loss: 0.0794 - val_mean_squared_error: 0.0783
Epoch 6/10
63954/63954 - 7s - loss: 0.0573 - mean_squared_error: 0.0561 - val_loss: 0.0805 - val_mean_squared_error: 0.0793
Epoch 7/10
63954/63954 - 7s - loss: 0.0564 - mean_squared_error: 0.0551 - val_loss: 0.0838 - val_mean_squared_error: 0.0825
Epoch 8/10
63954/63954 - 7s - loss: 0.0555 - mean_squared_error: 0.0542 - val_los

In [ ]:
cin_layer_size_dict

{(1, 1): {'RMSE': 0.292},
 (2, 2): {'RMSE': 0.292},
 (4, 4): {'RMSE': 0.292},
 (8, 8): {'RMSE': 0.289},
 (32, 32): {'RMSE': 0.288},
 (128, 128): {'RMSE': 0.291}}

### To get Best CIN_split half


In [ ]:
cin_split_half_dict = {}
for i in param_rand['cin_split_half']:
    model= xDeepFM(linear_feature_columns,dnn_feature_columns,dnn_hidden_units=(256, 256),
                    cin_layer_size=(128, 128),cin_activation='relu',cin_split_half=i,
                    seed=1024,task='binary')

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    cin_split_half_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['cin_split_half']=cin_split_half_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 57s - loss: 0.0780 - mean_squared_error: 0.0779 - val_loss: 0.0708 - val_mean_squared_error: 0.0704
Epoch 2/10
63954/63954 - 51s - loss: 0.0658 - mean_squared_error: 0.0653 - val_loss: 0.0726 - val_mean_squared_error: 0.0720
Epoch 3/10
63954/63954 - 51s - loss: 0.0625 - mean_squared_error: 0.0618 - val_loss: 0.0759 - val_mean_squared_error: 0.0750
Epoch 4/10
63954/63954 - 51s - loss: 0.0601 - mean_squared_error: 0.0591 - val_loss: 0.0791 - val_mean_squared_error: 0.0781
Epoch 5/10
63954/63954 - 51s - loss: 0.0585 - mean_squared_error: 0.0574 - val_loss: 0.0797 - val_mean_squared_error: 0.0786
Epoch 6/10
63954/63954 - 50s - loss: 0.0573 - mean_squared_error: 0.0561 - val_loss: 0.0816 - val_mean_squared_error: 0.0804
Epoch 7/10
63954/63954 - 50s - loss: 0.0563 - mean_squared_error: 0.0550 - val_loss: 0.0848 - val_mean_squared_error: 0.0834
Epoch 8/10
63954/63954 - 50s - loss: 0.0555 - mean_squared_error: 0.0542 - 

In [ ]:
cin_split_half_dict

{False: {'RMSE': 0.291}, True: {'RMSE': 0.289}}

### TO get best paramter l2_reg_CIN

In [ ]:
l2_reg_cin_dict={}
for i in param_rand['l2_reg_cin']:
    model = xDeepFM(linear_feature_columns,dnn_feature_columns,dnn_hidden_units=(256, 256),
                    cin_layer_size=(128, 128),cin_activation='relu',cin_split_half=True,l2_reg_cin=i,
                    seed=1024,task='binary')

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    l2_reg_cin_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
    
result['l2_reg_cin']=l2_reg_cin_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 58s - loss: 0.0781 - mean_squared_error: 0.0779 - val_loss: 0.0709 - val_mean_squared_error: 0.0706
Epoch 2/10
63954/63954 - 51s - loss: 0.0657 - mean_squared_error: 0.0652 - val_loss: 0.0723 - val_mean_squared_error: 0.0717
Epoch 3/10
63954/63954 - 51s - loss: 0.0623 - mean_squared_error: 0.0616 - val_loss: 0.0760 - val_mean_squared_error: 0.0752
Epoch 4/10
63954/63954 - 50s - loss: 0.0601 - mean_squared_error: 0.0591 - val_loss: 0.0779 - val_mean_squared_error: 0.0769
Epoch 5/10
63954/63954 - 51s - loss: 0.0585 - mean_squared_error: 0.0575 - val_loss: 0.0789 - val_mean_squared_error: 0.0778
Epoch 6/10
63954/63954 - 51s - loss: 0.0571 - mean_squared_error: 0.0560 - val_loss: 0.0828 - val_mean_squared_error: 0.0816
Epoch 7/10
63954/63954 - 51s - loss: 0.0560 - mean_squared_error: 0.0548 - val_loss: 0.0865 - val_mean_squared_error: 0.0852
Epoch 8/10
63954/63954 - 51s - loss: 0.0550 - mean_squared_error: 0.0537 - 

In [ ]:
l2_reg_cin_dict

{1e-07: {'RMSE': 0.29},
 1e-05: {'RMSE': 0.294},
 0.001: {'RMSE': 0.287},
 0.1: {'RMSE': 0.292},
 1: {'RMSE': 0.289}}

### To get best paramter l2_reg_linear

In [ ]:
l2_reg_linear_dict={}
for i in param_rand['l2_reg_linear']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), l2_reg_linear=i,
                    cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', l2_reg_cin=1e-07,
                    dnn_use_bn=False,seed=1024,task='binary')

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    l2_reg_linear_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
    
result['l2_reg_linear']=l2_reg_linear_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 59s - loss: 0.0781 - mean_squared_error: 0.0779 - val_loss: 0.0708 - val_mean_squared_error: 0.0704
Epoch 2/10
63954/63954 - 51s - loss: 0.0656 - mean_squared_error: 0.0651 - val_loss: 0.0725 - val_mean_squared_error: 0.0719
Epoch 3/10
63954/63954 - 51s - loss: 0.0623 - mean_squared_error: 0.0615 - val_loss: 0.0754 - val_mean_squared_error: 0.0745
Epoch 4/10
63954/63954 - 51s - loss: 0.0600 - mean_squared_error: 0.0591 - val_loss: 0.0797 - val_mean_squared_error: 0.0787
Epoch 5/10
63954/63954 - 51s - loss: 0.0586 - mean_squared_error: 0.0575 - val_loss: 0.0822 - val_mean_squared_error: 0.0810
Epoch 6/10
63954/63954 - 51s - loss: 0.0573 - mean_squared_error: 0.0562 - val_loss: 0.0818 - val_mean_squared_error: 0.0806
Epoch 7/10
63954/63954 - 52s - loss: 0.0563 - mean_squared_error: 0.0550 - val_loss: 0.0853 - val_mean_squared_error: 0.0839
Epoch 8/10
63954/63954 - 51s - loss: 0.0555 - mean_squared_error: 0.0541 - 

In [ ]:
l2_reg_linear_dict

{1e-05: {'RMSE': 0.294},
 0.001: {'RMSE': 0.292},
 0.1: {'RMSE': 0.29},
 1: {'RMSE': 0.29},
 10: {'RMSE': 0.291}}

### To get best paramter of l2_reg_embedding


In [ ]:
l2_reg_embedding_dict={}
for i in param_rand['l2_reg_embedding']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns,dnn_hidden_units=(256, 256), l2_reg_linear=0.1,
            l2_reg_embedding=i,cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', 
            l2_reg_cin=1e-07, seed=1024, task='binary',dnn_use_bn=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    l2_reg_embedding_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['l2_reg_embedding']=l2_reg_embedding_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 62s - loss: 0.0786 - mean_squared_error: 0.0777 - val_loss: 0.0715 - val_mean_squared_error: 0.0713
Epoch 2/10
63954/63954 - 52s - loss: 0.0651 - mean_squared_error: 0.0649 - val_loss: 0.0730 - val_mean_squared_error: 0.0728
Epoch 3/10
63954/63954 - 52s - loss: 0.0611 - mean_squared_error: 0.0609 - val_loss: 0.0760 - val_mean_squared_error: 0.0758
Epoch 4/10
63954/63954 - 52s - loss: 0.0588 - mean_squared_error: 0.0586 - val_loss: 0.0786 - val_mean_squared_error: 0.0784
Epoch 5/10
63954/63954 - 52s - loss: 0.0570 - mean_squared_error: 0.0568 - val_loss: 0.0801 - val_mean_squared_error: 0.0799
Epoch 6/10
63954/63954 - 52s - loss: 0.0558 - mean_squared_error: 0.0556 - val_loss: 0.0819 - val_mean_squared_error: 0.0817
Epoch 7/10
63954/63954 - 52s - loss: 0.0545 - mean_squared_error: 0.0543 - val_loss: 0.0833 - val_mean_squared_error: 0.0831
Epoch 8/10
63954/63954 - 52s - loss: 0.0535 - mean_squared_error: 0.0533 - 

In [ ]:
l2_reg_embedding_dict

{1e-07: {'RMSE': 0.293},
 1e-05: {'RMSE': 0.29},
 0.001: {'RMSE': 0.267},
 0.1: {'RMSE': 0.265},
 1: {'RMSE': 0.268}}

### To get best parameter l2-reg-dnn

In [ ]:
l2_reg_dnn_dict={}
for i in param_rand['l2_reg_dnn']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256), l2_reg_linear=0.1,
            cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', l2_reg_cin=1e-07,
            l2_reg_embedding=1e-05, l2_reg_dnn=i,seed=1024,task='binary',dnn_use_bn=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    l2_reg_dnn_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['l2_reg_dnn']=l2_reg_dnn_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 66s - loss: 0.0787 - mean_squared_error: 0.0777 - val_loss: 0.0709 - val_mean_squared_error: 0.0704
Epoch 2/10
63954/63954 - 54s - loss: 0.0658 - mean_squared_error: 0.0652 - val_loss: 0.0731 - val_mean_squared_error: 0.0723
Epoch 3/10
63954/63954 - 54s - loss: 0.0624 - mean_squared_error: 0.0615 - val_loss: 0.0761 - val_mean_squared_error: 0.0751
Epoch 4/10
63954/63954 - 54s - loss: 0.0603 - mean_squared_error: 0.0592 - val_loss: 0.0785 - val_mean_squared_error: 0.0774
Epoch 5/10
63954/63954 - 54s - loss: 0.0585 - mean_squared_error: 0.0574 - val_loss: 0.0809 - val_mean_squared_error: 0.0797
Epoch 6/10
63954/63954 - 54s - loss: 0.0575 - mean_squared_error: 0.0562 - val_loss: 0.0829 - val_mean_squared_error: 0.0816
Epoch 7/10
63954/63954 - 53s - loss: 0.0565 - mean_squared_error: 0.0551 - val_loss: 0.0850 - val_mean_squared_error: 0.0835
Epoch 8/10
63954/63954 - 54s - loss: 0.0556 - mean_squared_error: 0.0541 - 

### To get Best parameter dnn


In [ ]:
dnn_dropout_dict={}
for i in param_rand['dnn_dropout']:
    model = xDeepFM(linear_feature_columns, dnn_feature_columns, dnn_hidden_units=(256, 256),
                   cin_layer_size=(128, 128), cin_split_half=True, cin_activation='relu', l2_reg_cin=1e-07,
                   seed=1024, dnn_dropout=i, dnn_activation='relu',task='binary',dnn_use_bn=False)

    model.compile("adam", "mse", metrics=['mse'])

    history = model.fit(train_model_input, train[target].values,
                            batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
    pred_ans = model.predict(test_model_input, batch_size=256)
    
    auc = roc_auc_score(test[target].values, pred_ans)
    
    dnn_dropout_dict[i]={"RMSE": np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3)}
result['dnn_dropout']=dnn_dropout_dict

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 67s - loss: 0.0781 - mean_squared_error: 0.0780 - val_loss: 0.0708 - val_mean_squared_error: 0.0705
Epoch 2/10
63954/63954 - 54s - loss: 0.0657 - mean_squared_error: 0.0652 - val_loss: 0.0728 - val_mean_squared_error: 0.0721
Epoch 3/10
63954/63954 - 54s - loss: 0.0623 - mean_squared_error: 0.0615 - val_loss: 0.0772 - val_mean_squared_error: 0.0764
Epoch 4/10
63954/63954 - 54s - loss: 0.0601 - mean_squared_error: 0.0591 - val_loss: 0.0785 - val_mean_squared_error: 0.0775
Epoch 5/10
63954/63954 - 54s - loss: 0.0584 - mean_squared_error: 0.0574 - val_loss: 0.0786 - val_mean_squared_error: 0.0774
Epoch 6/10
63954/63954 - 55s - loss: 0.0574 - mean_squared_error: 0.0562 - val_loss: 0.0824 - val_mean_squared_error: 0.0812
Epoch 7/10
63954/63954 - 54s - loss: 0.0563 - mean_squared_error: 0.0550 - val_loss: 0.0858 - val_mean_squared_error: 0.0845
Epoch 8/10
63954/63954 - 54s - loss: 0.0553 - mean_squared_error: 0.0540 - 

### To train the model with best hyperparamter


In [22]:
model = xDeepFM(linear_feature_columns,dnn_feature_columns, dnn_hidden_units=(128,128), 
             seed=1024,task='binary')

model.compile("adam", "mse", metrics=['mse'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [25]:
history = model.fit(train_model_input, train[target].values,
                        batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

Train on 63954 samples, validate on 15989 samples
Epoch 1/10
63954/63954 - 43s - loss: 0.0546 - mean_squared_error: 0.0531 - val_loss: 0.0824 - val_mean_squared_error: 0.0809
Epoch 2/10
63954/63954 - 43s - loss: 0.0536 - mean_squared_error: 0.0521 - val_loss: 0.0858 - val_mean_squared_error: 0.0842
Epoch 3/10
63954/63954 - 43s - loss: 0.0532 - mean_squared_error: 0.0516 - val_loss: 0.0835 - val_mean_squared_error: 0.0819
Epoch 4/10
63954/63954 - 43s - loss: 0.0525 - mean_squared_error: 0.0509 - val_loss: 0.0867 - val_mean_squared_error: 0.0850
Epoch 5/10
63954/63954 - 43s - loss: 0.0521 - mean_squared_error: 0.0504 - val_loss: 0.0863 - val_mean_squared_error: 0.0847
Epoch 6/10
63954/63954 - 45s - loss: 0.0516 - mean_squared_error: 0.0500 - val_loss: 0.0864 - val_mean_squared_error: 0.0848
Epoch 7/10
63954/63954 - 43s - loss: 0.0509 - mean_squared_error: 0.0493 - val_loss: 0.0897 - val_mean_squared_error: 0.0881
Epoch 8/10
63954/63954 - 43s - loss: 0.0503 - mean_squared_error: 0.0486 - 

In [26]:
pred_ans = model.predict(test_model_input, batch_size=256)

auc = roc_auc_score(test[target].values, pred_ans)

In [27]:
print("RMSE:\t%f" % np.round(math.sqrt(mean_squared_error(test[target].values, pred_ans)),3),
      "MAE:\t%f" % np.round(mean_absolute_error(test[target].values, pred_ans),3),
      sep='\n')

RMSE:	0.312000
MAE:	0.137000
